### Import libraries

In [426]:
import hdlparse.verilog_parser as vlog
import re
import random

### open and read verilog file

In [427]:
vlog_ex = vlog.VerilogExtractor()
fname = 'Module1.v'
with open(fname, 'rt') as fh:
  code = fh.read()
vlog_mods = vlog_ex.extract_objects_from_source(code)

vlog_mods = vlog_ex.extract_objects(fname)

In [450]:
if_cases_conditions = set()
always_conditions = set()
case_conditions = set()

### print module name and ports

In [428]:
for m in vlog_mods:
  print('Module "{}":'.format(m.name))

  # print('  Parameters:')
  # for p in m.generics:
  #   print('\t{:20}{:8}{}'.format(p.name, p.mode, p.data_type))

  print('  Ports:')
  for p in m.ports:
    print('\t{:20}{:8}{}'.format(p.name, p.mode, p.data_type))

Module "Test1":
  Ports:
	HasAccess           input   wire
	Input_Pin           input   reg [3:0]
	message             output  reg [3:0]
	Welcome             output  reg


### add each inputs and outputs in list as dictionary

In [429]:
for m in vlog_mods:

  inputs_list = []
  ports_list = []
  print('\n  Ports:')
  # for p in m.ports:
  #   inputs_list.append({"mode":p.mode, "type":p.data_type, "name":p.name})
  #   ports_list.append((p.mode + ' ' + p.data_type + ' ' + p.name))
  #   print('\t\t',(p.mode + ' ' + p.data_type + ' ' + p.name))
  for p in m.ports:
        if (p.data_type).find('[') != -1 :
            inputs_list.append({"name":p.name ,"mode":p.mode ,"data_type":(p.data_type).split()[0],"length":vector_size((p.data_type).split()[1]) })
        else :
            inputs_list.append({"name":p.name ,"mode":p.mode ,"data_type":p.data_type, "length": 1})
        print('\t{:20}{:8}{}'.format(p.name, p.mode, p.data_type))



  Ports:
	HasAccess           input   wire
	Input_Pin           input   reg [3:0]
	message             output  reg [3:0]
	Welcome             output  reg


In [451]:
print(inputs_list)

[{'name': 'HasAccess', 'mode': 'input', 'data_type': 'wire', 'length': 1}, {'name': 'Input_Pin', 'mode': 'input', 'data_type': 'reg', 'length': 4}, {'name': 'message', 'mode': 'output', 'data_type': 'reg', 'length': 4}, {'name': 'Welcome', 'mode': 'output', 'data_type': 'reg', 'length': 1}]


In [413]:
for m in vlog_mods:
  parameters_list = []
  # ports_list = []
  print('\nModule "{}":'.format(m.name))

  print('  Parameters:')
  for p in m.generics:
    print('\t\t',(p.mode + ' ' +  p.data_type + ' ' + p.name))
    parameters_list.append(p.mode + ' ' + p.data_type + ' ' + p.name)

  print('\n  Ports:')
  for p in m.ports:
    # ports_list.append((p.mode + ' ' + p.data_type + ' ' + p.name))
    print('\t\t',(p.mode + ' ' + p.data_type + ' ' + p.name))


Module "Test1":
  Parameters:

  Ports:
		 input wire HasAccess
		 input reg [3:0] Input_Pin
		 output reg [3:0] message
		 output reg Welcome


## Control flow parsing

### parseing if conditions

In [452]:
def parse_if(line):
    if line.find("||") != -1 or line.find("&&") != -1:
        for port in inputs_list :
            if port["mode"]=="input" :
                if line.find(port["name"]) != -1 :
                    if_cases_conditions.add(port["name"])
    else :
        str = re.split("if\s*\(\s*!?",line)[1]
        str = re.split("\s*\)",str)[0]
        if_cases_conditions.add(str)

In [463]:
str = "if  (HasAccess || b)"
print((re.split("if\s*\(\s*!?",str))[1])
# parse_if(str)
# print(if_cases_conditions)

HasAccess || b)


### parseing always statement

In [465]:
def parse_always(line) :
    if line.find("||") != -1 or line.find("&&") != -1 or line.find("or") != -1 or line.find("and") != -1:
        for port in inputs_list :
            if port["mode"]=="input" :
                if line.find(port["name"]) != -1 :
                    always_conditions.add(port["name"])
    else :
        str = re.split("always\s*@\s*\(\s*!?",line)[1]
        str = re.split("\s*\)",str)[0]
        always_conditions.add(str)

### parseing case statemnt

In [466]:
def parse_case(line):
    str = re.split("case\s*\(\s*",line)[1]
    str = re.split("\s*\)",str)[0]
    case_conditions.add(str)

In [467]:
file = open(fname,"rt")
for line in file :
    if re.search("if\s*\(\s*\w+",line) :
        parse_if(line)
    if re.search("always\s*@\s*\(\s*\w+\s*",line):
        parse_always(line)
    if re.search("case\s*\(\s*\w+\s*\)",line) :
        parse_case(line)
print(if_cases_conditions)
print(always_conditions)
print(case_conditions)

{'x', "message == 4'b1111 & Input_Pin == pin", 'HasAccess'}
{'HasAccess'}
set()


### vector_size function return vector size or 1 if it not a vector

In [431]:
def vector_size(line):
	if re.search(r'\[', line):
		lval = re.findall(r'\[(.*):', line)[0]
		rval = re.findall(r'\[.*:(.*)\]', line)[0]
		return (abs(int(rval) - int(lval)) + 1)
	return 1

In [432]:
print(inputs_list[0]['mode'])

input


In [417]:
# print('inputs vector size')
# for i in range(len(inputs_list)):
#     for mode in inputs_list[i]['mode']:
#         if(mode.strip() == 'input'):
#             print('line ', i, '-> ', vector_size(line))

inputs vector size


### print testbench file in console

In [418]:
# time_delay = 10
# print('module ' + m.name + '_tb();')
# for port in ports_list:
    
#     if port.find('input', 0, 5) != -1:
#         print('    ', port.replace('input', 'reg'), ';')
#     elif port.find('output', 0, 6) != -1:
#         print('    ', port.replace('output', 'wire'),';')
#     # else:
#     #     print('    ', port, ';')
# print('//initial block\ninitial','\n  begin', '\n    //initial values')
# for i in range (7):
#     if(i != 0):
#         print('  #'+ str(time_delay))
#     for line in ports_list:
#         if line.split()[0] == 'input':
#             val = vector_size(line)
#             print('    ',line.split()[-1], '=', val,'\'b'+ bin(random.randint(0, pow(2,val)-1))[2:].zfill(val))

# print('  $finish;','\nend\n')
# print(m.name, 'DUT (')
# for i,line in enumerate(ports_list, 0):
#     print('  ',line.split()[-1], end='')
#     if i != len(ports_list)-1:
#         print(',')
#     else:
#         print('\n);')

# print('\nendmodule')


module Test1_tb();
//initial block
initial 
  begin 
    //initial values
  #10
  #10
  #10
  #10
  #10
  #10
  $finish; 
end

Test1 DUT (

endmodule


In [419]:
# old version
content = ""
time_delay = 10 #input or not
content += ('module ' + m.name + ' _tb();\n')
for port in ports_list:
    if port.find('wire'):
        port.replace('wire', ' ')
        if port.find('input', 0, 5) != -1:
            content += ('    ' + port.replace('input', 'reg') + ';\n')
        elif port.find('output', 0, 6) != -1:
            
            content += ('    ' + port.replace('output', 'wire') + ';\n')
        else:
            content += ('    ', port, ';\n')
content += ('//initial block\ninitial\n  begin\n    //initial values\n')
for i in range (8):
    if(i != 0):
        content += ('  #'+ str(time_delay)+'\n')
    for line in ports_list:
        if line.split()[0] == 'input':
            val = vector_size(line)
            content += ('    ' + line.split()[-1] + ' = '+ str(val) + '\'b'+ bin(random.randint(0, pow(2,val)-1))[2:].zfill(val)+'\n')

content += ('  $finish;\nend\n')
content += (m.name + ' DUT (\n')
for i,line in enumerate(ports_list, 0):
    content += ('  .' + (line.split()[-1]).replace('_tb', '') + '(' + line.split()[-1]+')')
    if i != len(ports_list)-1:
        content += (',\n')
    else:
        content += ('\n);\n')

content += ('\nendmodule')

### generate vector function generate the vector if size > 1

In [433]:
def generate_vector(port_length):
    if port_length == 1:
        return ' '
    else:
        return (' [' + str(port_length - 1) + ':0] ')

### add testbench content in string

In [434]:
def testbench_header_and_ports():
    content = ""
    content += ('module ' + m.name + '_tb();\n')
    for port in inputs_list:
    # if port["mode"].find('wire'):
    #     port.replace('wire', ' ')
        if port["mode"].find('input', 0, 5) != -1:
            content += ('    ' + port["mode"].replace('input', 'reg') +  generate_vector(port["length"]) + port["name"] + ';\n')
        elif port["mode"].find('output', 0, 6) != -1:
            content += ('    ' + port["mode"].replace('output', 'wire') + generate_vector(port["length"]) + port["name"] +  ';\n')
    return content
        # else:
        #     content += ('    ', port, ';\n')

In [435]:
def testbench_initial_block(time_delay):
    content = ""
    content += ('//initial block\ninitial\n  begin\n    //initial values\n')
    maxPortSize = 3
    for port in inputs_list :
        maxPortSize = max(port["length"],maxPortSize)
    iterationCounter = int(pow(2,maxPortSize)/2)
    for i in range (iterationCounter):
        if(i != 0):
            content += ('  #'+ str(time_delay)+'\n')
        for line in inputs_list:
            if line["mode"] == 'input':
                val = line["length"]
                content += ('    ' + line["name"] + ' = '+ str(val) + '\'b'+ bin(random.randint(0, pow(2,val)-1))[2:].zfill(val)+'\n')

    content += ('  $finish;\nend\n')
    return content

In [446]:
def testbench_design_instance():
    content = ""
    content += ('// instaniate design instance\n  ' + m.name + ' DUT (\n')
    for i,line in enumerate(inputs_list, 0):
        content += ('    .' + (line["name"]).replace('_tb', '') + '(' + line["name"]+')')
        if i != len(inputs_list)-1:
            content += (',\n')
        else:
            content += ('\n);\n')

    content += ('\nendmodule')

    return content

### testbench file generation

In [437]:
def generate_output_file(file_content):
    with open('output.v', 'w') as f:
        f.write(file_content)

### Testbench generator

In [449]:
time_delay = 10 #input or not
test_bench_content = ""
test_bench_content += testbench_header_and_ports() + testbench_initial_block(time_delay) + testbench_design_instance()
generate_output_file(test_bench_content)